互信息特征提取

In [1]:
import os
import glob
import pandas as pd
import networkx as nx
import numpy as np
import torch
import scipy.stats
from sklearn.preprocessing import MaxAbsScaler, KBinsDiscretizer

DATA_PATH = "D:/code/DTI_data/network_FN/"

def data_list(sample_path):
    sub_dirs = [x[0] for x in os.walk(sample_path)]
    sub_dirs.pop(0)

    data_list = []

    for sub_dir in sub_dirs:
        file_list = []
        dir_name = os.path.basename(sub_dir)
        file_glob = os.path.join(sample_path, dir_name, '*')
        file_list.extend(glob.glob(file_glob))

        for file_name in file_list:
            data_list.append([file_name, dir_name])

    return np.array(data_list)


class MRI_Dataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __getitem__(self, idx):
        filepath, target = self.data_list[idx][0], int(self.data_list[idx][1])
        dataframe = pd.read_csv(filepath, sep="\s+", header=None)
        pic = dataframe.to_numpy()

        return pic, target, idx

    def __len__(self):
        return len(self.data_list)

    
filelist = data_list(DATA_PATH)
dataset = MRI_Dataset(filelist)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=False)

for data, target, idx in dataloader:
    x = data.numpy()
    y = target.numpy()
    idx = idx.numpy()
    
x = x.reshape(x.shape[0], -1)

x0 = x[y == 0]
x1 = x[y == 1]
t_value, p_value = scipy.stats.ttest_ind(x0, x1, axis=0, equal_var=False, nan_policy='omit')
x_t = x[:, p_value <=0.05]
idx = np.where(p_value <=0.05)[0]
print(x_t.shape, idx)

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in less_equal
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in less_equal


(98, 104) [  10   74  185  187  188  226  246  279  294  452  632  722  813  900
  980 1008 1047 1215 1241 1302 1631 1823 1929 2011 2050 2090 2163 2667
 2691 2732 2812 2910 2947 3105 3286 3369 3506 3531 3547 3565 3599 3672
 3794 3825 3828 3915 3919 4063 4084 4092 4093 4116 4142 4176 4228 4297
 4362 4386 4453 4552 4673 4730 4946 4989 5141 5159 5308 5379 5917 5942
 5985 5988 6021 6062 6077 6095 6115 6269 6281 6322 6403 6459 6520 6657
 6660 6837 6924 7191 7210 7319 7356 7369 7466 7521 7636 7717 7778 7794
 7822 7903 7905 7966 7978 8049]


In [4]:
def Shannon_entropy(A):
    unique, counts = np.unique(A, return_counts=True)
    p = counts/counts.sum()
    ent = -np.sum(p * np.log2(p))
    return ent

def mutual_information(A, B):
    H_A = Shannon_entropy(A)
    unique, counts= np.unique(B, return_counts=True)
    H_A1B = 0
    for idx, status in enumerate(unique):
        H_A1B += Shannon_entropy(A[B==status]) * counts[idx]/counts.sum()
    MI_AB = H_A - H_A1B
    return MI_AB

In [11]:
import matplotlib.pyplot as plt

scaler = MaxAbsScaler()
scaler.fit(x_t)
xnew_train = scaler.transform(x_t)

est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
est.fit(xnew_train)
xnew_train = est.transform(xnew_train)

MI_array = np.zeros(x.shape[1])
for idx, e in enumerate(MI_array):
    MI_array[idx] = mutual_information(x[:, idx], y)
print(MI_array)
print(MI_array[0] == 0)
print(MI_array.shape, np.sum(MI_array>0.55))

[-0.          0.26558988  0.44916955 ...  0.31870478 -0.
 -0.        ]
True
(8100,) 118


In [12]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import KFold
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

seed = 1
cv = 20
kf = KFold(n_splits=cv, shuffle=True, random_state=seed)
acc_sum = 0
for idx, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    x_t = x[train_idx]
    y_t = y[train_idx]
    
    est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
    est.fit(x_t)
    x_d = est.transform(x_t)

    MI_array = np.zeros(x_d.shape[1])
    for idx, e in enumerate(MI_array):
        MI_array[idx] = mutual_information(x_d[:, idx], y_t)
    
    x_d = x[:, MI_array>0.55]

    ynew_train = y[train_idx]
    ynew_test = y[test_idx]


    # Norm
    scaler = MaxAbsScaler()
    scaler.fit(x_d[train_idx])
    xnew_train = scaler.transform(x_d[train_idx])
    xnew_test = scaler.transform(x_d[test_idx])
    print(xnew_train.shape[1])
    

    # SVC
    svc = SVC(kernel='rbf', random_state=1, gamma=0.01, C=10)
    model = svc.fit(xnew_train, ynew_train)

    predict = model.predict(xnew_test)
    correct = np.sum(predict == ynew_test)
    accuracy = correct / test_idx.size
    print("cv: {}/{}, acc.: {:.1f}\n".format(idx, cv, accuracy*100))
    acc_sum += accuracy
print("total acc.: {:.1f}\n".format(acc_sum / cv * 100))

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 0 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 4 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 5 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 9 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 11 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 94 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 95 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 96 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 98 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 99 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 177 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 178 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 182 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 189 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 197 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 306 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 310 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 311 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 312 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 314 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 409 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 411 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 413 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 415 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 416 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2

...

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7846 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7847 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7848 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7849 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7850 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\tor

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7914 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7916 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7917 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7918 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7921 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\tor

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7985 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7989 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7991 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7993 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 7995 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\tor

c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 8050 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 8054 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 8056 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 8058 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\torch1.2\lib\site-packages\sklearn\preprocessing\_discretization.py:159: UserWarning: Feature 8060 is constant and will be replaced with 0.
  "replaced with 0." % jj)
c:\users\shear\.conda\envs\tor

ValueError: Found array with 0 feature(s) (shape=(93, 0)) while a minimum of 1 is required by MaxAbsScaler.